# Analysis with SQL

SQL is a declarative programming language: (https://en.wikipedia.org/wiki/SQL).
With SQL, you declare what you want in a sentence that almost reads like English.

Today we are going to perform some exploratory analysis using both SQL and its equivalent in Pandas.


Instead of writing SQL on a DB specific platform we are going to make use of python library [**pandasql** ](https://pypi.org/project/pandasql/) which allows you to query pandas DataFrames using SQL syntax. It works similarly to sqldf in R. pandasql seeks to provide a more familiar way of manipulating and cleaning data for people new to Python or pandas.


We’re going to need a phrasebook!

### The anatomy of a SQL query
A SQL query consists of a few important keywords. Between those keywords, you add the specifics of what data, exactly, you want to see. Here is a skeleton query without the specifics:


SELECT… FROM… WHERE…


GROUP BY… HAVING…


ORDER BY…


LIMIT… OFFSET…


There are other terms, but these are the most important ones

**First Things First:**

Getting the data:-
    
Go to [dataset URL](https://ourairports.com/data/) and from the datasets download

**airports.csv** (8,635,267 bytes, last modified Sep 2, 2020)
Large file, containing information on all airports on this site.


**airport-frequencies.csv** (1,217,410 bytes, last modified Sep 2, 2020)
Large file, listing communication frequencies for the airports in airports.csv.


**runways.csv** (3,043,059 bytes, last modified Sep 2, 2020)
Large file, listing runways for the airports in airports.csv.

After downloading the data run the below cell to get the dependency-:

In [ ]:
pip install pandasql

In [31]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 02 23:55:34 2020

@author: PrakharJ

"""

import pandasql as ps
import pandas as pd

airports = pd.read_csv('data/airports.csv')
airport_freq = pd.read_csv('data/airport-frequencies.csv')
runways = pd.read_csv('data/runways.csv')

**SELECT, WHERE, DISTINCT, LIMIT**


Here are some SELECT statements. We truncate results with LIMIT, and filter them with WHERE. We use DISTINCT to remove duplicated results.

In [20]:
query1 = """select * from airports"""
query2 = """select * from airports limit 3"""
query3 = """select id from airports where ident = 'KLAX'"""
query4 = """select distinct type from airport"""

In [ ]:
ps.sqldf(query3, locals())

**SELECT with multiple conditions**


We join multiple conditions with an 'and'. If we only want a subset of columns from the table, then column subset is selected.


In [21]:
query5 = """select * from airports where iso_region = 'US-CA' and type = 'seaplane_base'"""

In [ ]:
ps.sqldf(query5, locals())

In [ ]:
query6 = """select ident, name, municipality from airports where iso_region = 'US-CA' and type = 'large_airport'"""

In [ ]:
ps.sqldf(query6, locals())

**ORDER BY**


We use order by to sort on column in ascending and order by desc to sort in descending

In [ ]:
query7 = """select * from airport_freq where airport_ident = 'KLAX' order by type"""

In [22]:
query8 = """select * from airport_freq where airport_ident = 'KLAX' order by type desc"""

In [ ]:
ps.sqldf(query8, locals())

**IN… NOT IN**


We know how to filter on a value, but what about a list of values — IN condition? 

In [ ]:
query9 = """select * from airports where type in ('heliport', 'balloonport')"""

In [ ]:
query10 = """select * from airports where type not in ('heliport', 'balloonport')"""

In [ ]:
ps.sqldf(query10, locals())

**GROUP BY, COUNT, ORDER BY**


Grouping is straightforward: use the .groupby() operator. There’s a subtle difference between semantics of a COUNT in SQL and Pandas. In Pandas, .count() will return the number of non-null/NaN values. To get the same result as the SQL COUNT, use .size().

In [ ]:
query11 = """select iso_country, type, count(*) from airports group by iso_country, type order by iso_country, type"""

In [ ]:
query12 = """select iso_country, type, count(*) from airports group by iso_country, type order by iso_country, count(*) desc"""

In [ ]:
ps.sqldf(query12, locals())

**HAVING**


In SQL, you can additionally filter grouped data using a HAVING condition. In Pandas, you can use .filter() and provide a Python function (or a lambda) that will return True if the group should be included into the result.


In [ ]:
query13 = """select type, count(*) from airports where iso_country = 'US' group by type having count(*) > 1000 order by count(*) desc"""

In [ ]:
ps.sqldf(query13, locals())

**Top N records**


Let’s say we did some preliminary querying, and now have a dataframe called by_country, that contains the number of airports per country:

In [ ]:
<img src="head_image.PNG">

In the next example, we order things by airport_count and only select the top 10 countries with the largest count. Second example is the more complicated case, in which we want “the next 10 after the top 10”:

In [ ]:
query14 = """select iso_country from by_country order by size desc limit 10"""

In [ ]:
query15 = """select iso_country from by_country order by size desc limit 10 offset 10"""

In [ ]:
ps.sqldf(query15, locals())

***Aggregate functions (MIN, MAX, MEAN)***


Now, given this dataframe or runway data:

<img src="runway_image.PNG">

Calculate min, max, mean, and median length of a runway:

In [ ]:
query16 = """select max(length_ft), min(length_ft), avg(length_ft), median(length_ft) from runways"""

In [ ]:
ps.sqldf(query16, locals())

***JOIN***


You need to provide which columns to join on (left_on and right_on), and join type: inner (default), left (corresponds to LEFT OUTER in SQL), right (RIGHT OUTER), or outer (FULL OUTER).

In [ ]:
query17 = """select airport_ident, type, description, frequency_mhz from airport_freq join airports on airport_freq.airport_ref = airports.id where airports.ident = 'KLAX'"""

In [ ]:
ps.sqldf(query17, locals())

**UNION ALL and UNION**


Use UNION ALL or To deduplicate things UNION:

In [ ]:
query18 = """select name, municipality from airports where ident = 'KLAX' union all select name, municipality from airports where ident = 'KLGB'"""

In [ ]:
ps.sqldf(query18, locals())

**To try out below queries use the link:**
    
[join](https://www.w3schools.com/sql/sql_join.asp#:~:text=Different%20Types%20of%20SQL%20JOINs&text=(INNER)%20JOIN%3A%20Returns%20records,records%20from%20the%20left%20table)

**INSERT**

In [26]:
query19 = """create table heroes (id integer, name text);"""
query20 = """insert into heroes values (1, 'Harry Potter');"""
query21 = """insert into heroes values (2, 'Ron Weasley');"""
query22 = """insert into heroes values (3, 'Hermione Granger');"""

**UPDATE**


Now we need to fix some bad data in the original dataframe:

In [32]:
query2 = """update heroes set name = 'dumbledore' where id == 2"""

**DELETE**

In [ ]:
query2 = """delete from heroes where id == 3"""